In [11]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

class Line():    
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = None
        #average x values of the fitted line over the last n iterations
        self.bestx = [[],[]] 
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = np.zeros((2, 3)) 
        #polynomial coefficients for the most recent fit
        self.current_fit = np.zeros((2, 3))
        #radius of curvature of the line in some units
        self.radius_of_curvature = np.zeros((2, 1))     
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = [np.array([0,0,0]),np.array([0,0,0])]
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None        
        #number of continuous error frames
        self.errorCount =0
        #to keep previous warp image
        self.newwarp=None
        #Curvature and offset
        self.curvature = None
        self.offset =None

    def calibration(self, path):
        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        objp = np.zeros((6*9,3), np.float32)
        objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)
        # Arrays to store object points and image points from all the images.
        objpoints = [] # 3d points in real world space
        imgpoints = [] # 2d points in image plane.
        # Make a list of calibration images
        images = glob.glob(path)
        retTrue_images=[]
        corner_images = []
        undist_images=[]
        counter =0
        # Step through the list and search for chessboard corners and undistort images
        for num in range(len(images)):
            img = cv2.imread(images[num])            
            img_size = (img.shape[1], img.shape[0])
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)    
            # Find the chessboard corners
            ret, corners = cv2.findChessboardCorners(gray, (9,6),None)     
            # If found, add object points, image points    
            if ret == True:                        
                objpoints.append(objp)
                imgpoints.append(corners)        
            else:
                counter+=1        
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
        self.mtx = mtx
        self.dist = dist
        print(str(counter)+" of "+str(len(images))+" failed to find corners for 9x6 chessboard")        

    def undistort(self, img):
        return cv2.undistort(img, self.mtx, self.dist, None, self.mtx)               
          
    def abs_sobel_thresh(self, img, orient='x', sobel_kernel=3, thresh=(0, 200)):    
        # Apply x or y gradient with the OpenCV Sobel() function
        # and take the absolute value
        if orient == 'x':
            abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 1, 0))
        if orient == 'y':
            abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 0, 1))
        # Rescale back to 8 bit integer
        scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
        # Create a copy and apply the threshold
        binary_output = np.zeros_like(scaled_sobel)
        # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
        binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

        # Return the result
        return binary_output

    def mag_thresh(self,img, sobel_kernel=3, mag_thresh=(0, 255)):    
        # Take both Sobel x and y gradients
        sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
        sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
        # Calculate the gradient magnitude
        gradmag = np.sqrt(sobelx**2 + sobely**2)
        # Rescale to 8 bit
        scale_factor = np.max(gradmag)/255 
        gradmag = (gradmag/scale_factor).astype(np.uint8) 
        # Create a binary image of ones where threshold is met, zeros otherwise
        binary_output = np.zeros_like(gradmag)
        binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

        # Return the binary image
        return binary_output

    def dir_threshold(self,img, sobel_kernel=3, thresh=(0, np.pi/2)):    
        # Calculate the x and y gradients
        sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
        sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
        # Take the absolute value of the gradient direction, 
        # apply a threshold, and create a binary image result
        absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
        binary_output =  np.zeros_like(absgraddir)
        binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

        # Return the binary image
        return binary_output

    def s_threshold(self,img, thresh=(0,255)):
        binary_output = np.zeros_like(img)
        binary_output[(img >= thresh[0]) & (img <= thresh[1])] = 1
        return binary_output

    def binary(self, img):
        ksize = 7
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HLS).astype(np.float)
        l_channel = hsv[:,:,1]
        s_channel = hsv[:,:,2]
        gradx = self.abs_sobel_thresh(l_channel, orient='x', sobel_kernel=ksize, thresh=(55, 100))
        grady = self.abs_sobel_thresh(l_channel, orient='y', sobel_kernel=ksize, thresh=(30, 100))
        mag_binary = self.mag_thresh(l_channel, sobel_kernel=ksize, mag_thresh=(30, 100))
        dir_binary = self.dir_threshold(l_channel, sobel_kernel=ksize, thresh=((0.8, 1.1)))
        s_binary = self.s_threshold(s_channel, thresh=((170,215)))
        combined = np.zeros_like(l_channel)
        combined[((gradx == 1) & (grady == 1)) |
                 ((mag_binary == 1) & (dir_binary == 1)) |
                 (s_binary==1)] = 1    
        return combined    
    
    def transform(self, img):
        img_size = (img.shape[1], img.shape[0])
        src = np.float32([[592,450],[260,680],[1057,680],[691,450]])
        dst = np.float32([[(img_size[0] / 4), 0],
                          [(img_size[0] / 4), img_size[1]],
                          [(img_size[0] * 3 / 4), img_size[1]],
                          [(img_size[0] * 3 / 4), 0]])
        M = cv2.getPerspectiveTransform(src, dst)
        return cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    def findLane(self, binary_warped):            
        # Create an output image to draw on and  visualize the result        
        out_img = np.dstack((binary_warped, binary_warped, binary_warped)) 
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])  
        # Set the width of the windows +/- margin
        margin = 100

        if self.detected == True:
            left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
            right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  
        else:
            self.errorCount+=1
            # Take a histogram of the bottom half of the image
            print(int(binary_warped.shape[0]/2))
            histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)    
            # Find the peak of the left and right halves of the histogram
            # These will be the starting point for the left and right lines
            midpoint = np.int(histogram.shape[0]/2)
            leftx_base = np.argmax(histogram[:midpoint])
            rightx_base = np.argmax(histogram[midpoint:]) + midpoint
            # Choose the number of sliding windows
            nwindows = 9
            # Set height of windows
            window_height = np.int(binary_warped.shape[0]/nwindows)
            # Current positions to be updated for each window
            leftx_current = leftx_base
            rightx_current = rightx_base

            # Set minimum number of pixels found to recenter window
            minpix = 50
            # Create empty lists to receive left and right lane pixel indices
            left_lane_inds = []
            right_lane_inds = []

            # Step through the windows one by one
            for window in range(nwindows):
                # Identify window boundaries in x and y (and right and left)
                win_y_low = binary_warped.shape[0] - (window+1)*window_height
                win_y_high = binary_warped.shape[0] - window*window_height
                win_xleft_low = leftx_current - margin
                win_xleft_high = leftx_current + margin
                win_xright_low = rightx_current - margin
                win_xright_high = rightx_current + margin
                # Draw the windows on the visualization image
                #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,1,0), 2) 
                #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,1,0), 2) 
                # Identify the nonzero pixels in x and y within the window
                good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
                good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
                # Append these indices to the lists
                left_lane_inds.append(good_left_inds)
                right_lane_inds.append(good_right_inds)
                # If you found > minpix pixels, recenter next window on their mean position
                if len(good_left_inds) > minpix:
                    leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
                if len(good_right_inds) > minpix:        
                    rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            # Concatenate the arrays of indices
            left_lane_inds = np.concatenate(left_lane_inds)
            right_lane_inds = np.concatenate(right_lane_inds)
            self.detected == True

        #x,y values for detected line pixels                
        self.allx = [np.array(nonzerox[left_lane_inds]),np.array(nonzerox[right_lane_inds])]
        self.ally = [np.array(nonzeroy[left_lane_inds]),np.array(nonzeroy[right_lane_inds])]
        #polynomial coefficients for the most recent fit / 0:left, 1:right
        self.current_fit[0] = np.polyfit(self.ally[0],self.allx[0], 2)        
        self.current_fit[1] = np.polyfit(self.ally[1],self.allx[1], 2)
        
        # x values of the last n fits of the line
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
   
        #average x values of the fitted line over the last n iterations  
        n = 15 #Number of the iteration to average the array                
        if self.recent_xfitted is not None:            
            if self.recent_xfitted[0].shape[0] == n:                
                self.recent_xfitted[0] = np.delete(self.recent_xfitted[0],0,axis=0)
                self.recent_xfitted[1] = np.delete(self.recent_xfitted[1],0,axis=0)
            self.recent_xfitted[0] = np.vstack((self.recent_xfitted[0],
                       (self.current_fit[0][0]*ploty**2 + 
                       self.current_fit[0][1]*ploty + 
                       self.current_fit[0][2])))
            self.recent_xfitted[1] = np.vstack((self.recent_xfitted[1],
                       (self.current_fit[1][0]*ploty**2 + 
                       self.current_fit[1][1]*ploty + 
                       self.current_fit[1][2])))
                             
            self.bestx = [np.mean(self.recent_xfitted[0],axis=0),np.mean(self.recent_xfitted[1],axis=0)]
        else:
            self.recent_xfitted = [((np.array(self.current_fit[0][0]*ploty**2 + 
                                             self.current_fit[0][1]*ploty + 
                                             self.current_fit[0][2]))),
                                   ((np.array(self.current_fit[1][0]*ploty**2 + 
                                             self.current_fit[1][1]*ploty + 
                                             self.current_fit[1][2])))]
            self.bestx = [self.recent_xfitted[0],self.recent_xfitted[1]]
           
        
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = [np.polyfit(ploty,self.bestx[0], 2),np.polyfit(ploty,self.bestx[1], 2)        ]
        
        #difference in fit coefficients between last and new fits
        self.diffs = [abs(np.subtract(self.best_fit[0],self.current_fit[0])),
                      abs(np.subtract(self.best_fit[1],self.current_fit[1]))]     
        
        self.measure_curvature_offset(ploty)
                
        # Sanity Check
        if self.sanityCheck()==True:
            out_img[self.ally[0], self.allx[0]] = [1, 0, 0]
            out_img[self.ally[1], self.allx[1]] = [0, 0, 1]
            color_warp = np.zeros_like(out_img).astype(np.uint8)
            
            # Recast the x and y points into usable format for cv2.fillPoly()
            pts_left = np.array([np.transpose(np.vstack([self.bestx[0], ploty]))])
            pts_right = np.array([np.flipud(np.transpose(np.vstack([self.bestx[1], ploty])))])
            pts = np.hstack((pts_left, pts_right))

            # Draw the lane onto the warped blank image
            cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
            color_warp = cv2.addWeighted(color_warp,1,(out_img*255).astype(np.uint8),1,0)

            # Warp the blank back to original image space using inverse perspective matrix (Minv)
            self.newwarp = self.inv_transform(color_warp)
            
            # Combine the result with the original image
            result = cv2.addWeighted(cv2.cvtColor(self.undistort_img, cv2.COLOR_BGR2RGB), 1, self.newwarp, 0.3, 0)
        
            # Information box            
            self.curvature = 'Radius of Curvature = '+str(np.round(np.mean([self.radius_of_curvature[0],self.radius_of_curvature[1]]),1))+'(m)'
            
            if self.line_base_pos>=0:
                self.offset = 'Vehicle is '+str(self.line_base_pos)+'m right of the center'
            else:
                self.offset = 'Vehicle is '+str(-self.line_base_pos)+'m left of the center'

            cv2.rectangle(result, (8,25),(600,90), (0,0,0), -1)
            cv2.putText(result,self.curvature,(10,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
            cv2.putText(result,self.offset,(10,80), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)                                    
            return result        
        else:
            self.detected == False            
            #remove last fitted values from the bestx array unless there are no values, 720 means (720,) - one row
            if self.recent_xfitted[0].shape[0]!=720:
                self.recent_xfitted[0] = np.delete(self.recent_xfitted[0],-1,axis=0)
                self.recent_xfitted[1] = np.delete(self.recent_xfitted[1],-1,axis=0)                        
            self.errorCount+=1
            if self.errorCount ==1:
                self.findLane(binary_warped)
            # Less than 10 error frames / keep previous information
            elif self.errorCount <10: 
                if self.newwarp==None:
                    result = cv2.cvtColor(self.undistort_img, cv2.COLOR_BGR2RGB)
                else:
                    result = cv2.addWeighted(cv2.cvtColor(self.undistort_img, cv2.COLOR_BGR2RGB), 1, self.newwarp, 0.3, 0)                                
                cv2.rectangle(result, (8,25),(600,90), (0,0,0), -1)
                cv2.putText(result,self.curvature,(10,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
                cv2.putText(result,self.offset,(10,80), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)                                    
                return result
            # More than 10 error frames / show error / initialize previous x values
            else : 
                self.errorCount = 0
                self.recent_xfitted = None
                self.diffs = [np.array([0,0,0]),np.array([0,0,0])]
                if self.newwarp==None:
                    result = cv2.cvtColor(self.undistort_img, cv2.COLOR_BGR2RGB)
                else:
                    result = cv2.addWeighted(cv2.cvtColor(self.undistort_img, cv2.COLOR_BGR2RGB), 1, self.newwarp, 0.3, 0)                                
                self.curvature = "Cannot find lane"
                cv2.rectangle(result, (8,25),(600,90), (0,0,0), -1)                
                cv2.putText(result,self.curvature,(10,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)                
                return result
                      
    def measure_curvature_offset(self, ploty):
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension
        y_eval = np.max(ploty)

        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(ploty*ym_per_pix, self.bestx[0]*xm_per_pix, 2)
        right_fit_cr = np.polyfit(ploty*ym_per_pix, self.bestx[1]*xm_per_pix, 2)
        
        # Calculate the new radii of curvature
        self.radius_of_curvature[0] = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        self.radius_of_curvature[1] = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])            
        self.line_base_pos = np.round(((((self.bestx[1][0]+self.bestx[0][0])/2)-(self.undistort_img.shape[1]/2))*xm_per_pix),1)        

    def inv_transform(self, img):
        img_size = (img.shape[1], img.shape[0])
        dst = np.float32([[592,450],[260,680],[1057,680],[691,450]])
        src = np.float32([[(img_size[0] / 4), 0],
                          [(img_size[0] / 4), img_size[1]],
                          [(img_size[0] * 3 / 4), img_size[1]],
                          [(img_size[0] * 3 / 4), 0]])
        M = cv2.getPerspectiveTransform(src, dst)
        return cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    def sanityCheck(self):
        straight_flag = False
        #Checking that they have similar curvature
        if self.radius_of_curvature[0]>3000 or self.radius_of_curvature[0]>3000:
            # Since straight lane is very sensetive, ignore big value
            straight_flag = True            
        elif abs(self.radius_of_curvature[0]-self.radius_of_curvature[1])>500:
            print("Both curvature are not similar")
            print("Left: "+str(self.radius_of_curvature[0]))
            print("Right: "+str(self.radius_of_curvature[1]))
            return False
                    
        #Checking that they are separated by approximately the right distance horizontally
        if abs(self.line_base_pos)>3:
            print("Line_base_pos is separated from the center more than 3")
            print("Base position: "+str(self.line_base_pos))            
            return False
        
        #Checking that they are roughly parallel        
        if straight_flag == False:                    
            if self.diffs[0][1]-self.diffs[0][1]>5:
                pritn("Both lanes are not parallel")            
                print("Left difference: "+self.diffs[0])
                print("Right difference: "+self.diffs[1])
                return False        
        
        return True  
    
    #batch method
    def start(self, img):         
        self.undistort_img = self.undistort(img)
        binary = self.binary(self.undistort_img)
        binary_warped = self.transform(binary)
        result = self.findLane(binary_warped)
        return cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

In [9]:
import pickle

line = Line()
line.calibration('camera_cal/calibration*.jpg')

data = {'mtx':line.mtx,'dist':line.dist}

with open('mtx_dist.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

3 of 20 failed to find corners for 9x6 chessboard
[[  1.15396093e+03   0.00000000e+00   6.69705357e+02]
 [  0.00000000e+00   1.14802496e+03   3.85656234e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
[[ -2.41017956e-01  -5.30721173e-02  -1.15810355e-03  -1.28318856e-04
    2.67125290e-02]]


In [10]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
clip = VideoFileClip("project_video.mp4")
output ="project_video_lane.mp4"
line = Line()
with open('mtx_dist.pickle', 'rb') as handle:
    data = pickle.load(handle)
line.mtx, line.dist = data['mtx'], data['dist']
output_clip = clip.fl_image(line.start) 
%time output_clip.write_videofile(output, audio=False)

[[  1.15396093e+03   0.00000000e+00   6.69705357e+02]
 [  0.00000000e+00   1.14802496e+03   3.85656234e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
[[ -2.41017956e-01  -5.30721173e-02  -1.15810355e-03  -1.28318856e-04
    2.67125290e-02]]


TypeError: slice indices must be integers or None or have an __index__ method

In [12]:
from moviepy.editor import VideoFileClip

line = Line()
clip = VideoFileClip("challenge_video.mp4")
output ="challenge_video_lane.mp4"
line.calibration('camera_cal/calibration*.jpg')
output_clip = clip.fl_image(line.start) 
%time output_clip.write_videofile(output, audio=False)

3 of 20 failed to find corners for 9x6 chessboard
360
[MoviePy] >>>> Building video challenge_video_lane.mp4
[MoviePy] Writing video challenge_video_lane.mp4


  0%|          | 1/485 [00:00<03:30,  2.29it/s]

360


  0%|          | 2/485 [00:00<03:16,  2.45it/s]

360


...

100%|█████████▉| 484/485 [02:44<00:00,  3.48it/s]

360
Both curvature are not similar
Left: [ 174.18590315]
Right: [ 729.92317544]


100%|██████████| 485/485 [02:45<00:00,  3.51it/s]

360
Both curvature are not similar
Left: [ 174.18590315]
Right: [ 729.92317544]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_lane.mp4 

CPU times: user 3min 21s, sys: 8.93 s, total: 3min 30s
Wall time: 2min 46s


In [486]:
line = Line()
clip = VideoFileClip("harder_challenge_video.mp4")
output ="harder_challenge_video_lane.mp4"
line.mtx = mtx
line.dist = dist
output_clip = clip.fl_image(line.start) 
%time output_clip.write_videofile(output, audio=False)

Both curvature are not similar
Left: [ 166.52379924]
Right: [ 2256.11526374]
[MoviePy] >>>> Building video harder_challenge_video_lane.mp4
[MoviePy] Writing video harder_challenge_video_lane.mp4












  0%|          | 0/1200 [00:00<?, ?it/s]









  0%|          | 1/1200 [00:00<06:57,  2.87it/s]

Both curvature are not similar
Left: [ 166.52379924]
Right: [ 2256.11526374]












  0%|          | 2/1200 [00:00<07:14,  2.76it/s]

Both curvature are not similar
Left: [ 173.18167138]
Right: [ 1442.13893607]


...











100%|█████████▉| 1198/1200 [08:18<00:01,  1.79it/s]

Both curvature are not similar
Left: [ 64.19824072]
Right: [ 1948.82261292]












100%|█████████▉| 1199/1200 [08:19<00:00,  1.75it/s]

Both curvature are not similar
Left: [ 479.73376723]
Right: [ 2046.97371916]


[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_challenge_video_lane.mp4 

CPU times: user 8min 43s, sys: 1min 12s, total: 9min 55s
Wall time: 8min 22s
